# Meta Prompting

Meta prompting is an advanced technique in prompt engineering that emphasizes the structural and syntactical organization of tasks and problems rather than focusing on their specific content. The objective is to create a more abstract, form-driven way of engaging with large language models (LLMs), highlighting patterns and structure over traditional content-focused methods.

As outlined by [Zhang et al. (2024)](https://arxiv.org/abs/2311.11482), the defining features of meta prompting include:

* Structure-Oriented: Prioritizes the organization and pattern of problems and solutions instead of specific content.
* Syntax-Guided: Leverages syntax as a template to shape the expected responses or solutions.
* Abstract Frameworks: Uses abstract examples as blueprints, demonstrating the structure of tasks without relying on concrete details.
* Domain Versatility: Can be applied across multiple fields, offering structured solutions to diverse problem types.
* Categorical Approach: Draws on type theory to organize and categorize components logically, enhancing prompt coherence and precision.

## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Fmeta.ipynb)



In [2]:
pip install pandas


   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------------ --------------------------- 3.7/11.5 MB 23.0 MB/s eta 0:00:01
   --------------------------------- ------ 9.7/11.5 MB 23.7 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 20.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------- ----------------------- 5.2/12.6 MB 25.6 MB/s eta 0:00:01
   ------------------------------- -------- 10.0/12.6 MB 24.5 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 22.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
############################################################
## META ANALYSIS: COMPARE TECHNIQUES & OUTPUTS
############################################################

import os
import pandas as pd
import csv
from datetime import datetime
from _pipeline import create_payload, model_req

# 1) Paths to your log files
ZERO_SHOT_LOG = "data/zero_shot_logs.csv"
FEW_SHOT_LOG  = "data/few_shot_logs.csv"
COT_LOG       = "data/chain_of_thought_logs.csv"
SC_LOG        = "data/self_consistency_logs.csv"

def load_logs(log_path):
    """Load CSV logs if the file exists, otherwise return an empty DataFrame."""
    if os.path.exists(log_path):
        return pd.read_csv(log_path)
    else:
        return pd.DataFrame()

# 2) Load partial outputs from each technique
df_zero = load_logs(ZERO_SHOT_LOG)
df_few  = load_logs(FEW_SHOT_LOG)
df_cot  = load_logs(COT_LOG)
df_sc   = load_logs(SC_LOG)

# 3) Function to extract a representative or final sample output
def get_sample_output(df, technique_name):
    """
    Returns a short label plus a snippet of the technique's output.
    For demonstration, we grab the last row's 'response' or 'final_output' column.
    """
    if df.empty:
        return f"**{technique_name}:** No logs found.\n"
    row = df.iloc[-1]
    output_text = row.get("final_output") or row.get("response", "No output")
    # Truncate the output for the meta prompt if it's too long
    snippet = (output_text[:400] + "...") if len(output_text) > 400 else output_text
    return f"**{technique_name}** (Sample):\n{snippet}\n"

# 4) Build a meta-analysis prompt
zero_snippet = get_sample_output(df_zero, "Zero-Shot")
few_snippet  = get_sample_output(df_few,  "Few-Shot")
cot_snippet  = get_sample_output(df_cot,  "Chain-of-Thought")
sc_snippet   = get_sample_output(df_sc,   "Self-Consistency")

META_PROMPT = f"""

{zero_snippet}
{few_snippet}
{cot_snippet}
{sc_snippet}
# Instructions for Enhanced Meta Analysis of Technique Outputs

You are a senior technical analyst and report architect tasked with conducting a comprehensive meta analysis of the outputs generated by various report refinement techniques (including Zero-Shot, Few-Shot, Chain-of-Thought, Self-Consistency, and Final Merge). Your goal is to evaluate the overall effectiveness of each technique, compare their strengths and weaknesses, and provide actionable recommendations for further optimization of the report generation pipeline.

Follow these detailed steps:

1. **Review and Catalog Technique Outputs:**  
   - Examine the outputs produced by each technique:
     - **Zero-Shot Output:** The initial basic skeleton report outlining the fundamental structure.
     - **Few-Shot Output:** The refined version using example inputs to improve formatting, clarity, and detail.
     - **Chain-of-Thought Output:** The internally reasoned and comprehensive refinement of complex sections (e.g., Data & Methodologies), leading to an enriched final version.
     - **Self-Consistency Output:** Multiple candidate versions of challenging sections (e.g., Results & Evaluation), internally compared and refined to select the best version.
     - **Final Merge Output:** The fully integrated, cohesive, and polished project report in Markdown.

2. **Establish Evaluation Criteria:**  
   Assess each technique based on the following dimensions:
   - **Clarity and Readability:**  
     - How effectively does the output communicate the intended content?
     - Is the language clear and accessible to a technical audience?
   - **Comprehensiveness:**  
     - Does the output cover all required sections and details?
     - Are key elements such as the project overview, methodology, results, and conclusions fully developed?
   - **Logical Structure and Flow:**  
     - Is there a coherent progression of ideas across sections?
     - Are headings, subheadings, and transitions effectively used to guide the reader?
   - **Visual and Technical Presentation:**  
     - Is Markdown formatting consistent and visually appealing?
     - Are code blocks, visual placeholders, and tables used appropriately to enhance understanding?
   - **Innovation and Improvement:**  
     - How has each technique contributed to refining the report quality?
     - Are there innovative aspects or improvements that can be highlighted?

3. **Synthesize and Compare Findings:**  
   - Summarize the key strengths and limitations of each technique.
   - Identify any recurring challenges or notable successes across the outputs.
   - Determine which elements from each technique could be combined or further optimized to improve future iterations of the report generation process.

4. **Develop Actionable Recommendations:**  
   - Based on your analysis, propose specific strategies to enhance the pipeline.  
     For example:
       - Integrating the clarity of Few-Shot outputs with the detailed rigor of Chain-of-Thought outputs.
       - Refining visual formatting standards to ensure consistent presentation across sections.
       - Enhancing the internal evaluation criteria used during Self-Consistency to further improve candidate selection.
   - Provide insights on potential next steps for iterative improvements.

5. **Generate the Final Meta Analysis Report:**  
   - Prepare a concise, well-structured meta analysis report in Markdown that details your findings, comparisons, and recommendations.
   - Ensure that the report is professional, clear, and actionable, targeting both technical and managerial audiences.
   - Do not include any internal chain-of-thought details or expose the alternative candidate outputs—only the final synthesized insights should be shared.

**Important:**  
Your final output should consist solely of the enhanced meta analysis report in Markdown format. Internal reasoning or multiple candidate outputs should not be exposed.

Generate your final meta analysis report in Markdown as your output.

"""

# 5) Create the payload for meta analysis
payload_meta = create_payload(
    target="open-webui",
    model="qwen2",  # Example model for meta analysis
    prompt=META_PROMPT,
    temperature=0.7,
    num_ctx=250,
    num_predict=300
)

# 6) Send the request
time_meta, response_meta = model_req(payload=payload_meta)

# 7) Print the meta-analysis
print("===== META ANALYSIS OUTPUT =====")
print(response_meta)
if time_meta:
    print(f"\nTime taken for meta analysis: {time_meta}s")

# 8) Save meta analysis to a Markdown file
os.makedirs("outputs", exist_ok=True)
with open("outputs/meta_analysis.md", "w", encoding="utf-8") as f:
    f.write(response_meta)

# 9) (Optional) Log the meta-analysis result in a CSV
os.makedirs("data", exist_ok=True)
log_file = "data/meta_analysis_logs.csv"
with open(log_file, "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # Optionally write a header row if you haven't before:
    # writer.writerow(["timestamp", "model", "time_taken", "analysis_output"])
    writer.writerow([
        datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "qwen2",
        time_meta,
        response_meta.replace("\n", "\\n")
    ])


Payload: {'model': 'qwen2', 'messages': [{'role': 'user', 'content': '\n\n**Zero-Shot** (Sample):\nNo output\n\n**Few-Shot** (Sample):\nNo output\n\n**Chain-of-Thought** (Sample):\nNo output\n\n**Self-Consistency:** No logs found.\n\n# Instructions for Enhanced Meta Analysis of Technique Outputs\n\nYou are a senior technical analyst and report architect tasked with conducting a comprehensive meta analysis of the outputs generated by various report refinement techniques (including Zero-Shot, Few-Shot, Chain-of-Thought, Self-Consistency, and Final Merge). Your goal is to evaluate the overall effectiveness of each technique, compare their strengths and weaknesses, and provide actionable recommendations for further optimization of the report generation pipeline.\n\nFollow these detailed steps:\n\n1. **Review and Catalog Technique Outputs:**  \n   - Examine the outputs produced by each technique:\n     - **Zero-Shot Output:** The initial basic skeleton report outlining the fundamental struc